In [1]:
# from transformers import ViTFeatureExtractor

# # model_name_or_path = 'google/vit-base-patch16-224-in21k'
# # feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

In [2]:
# from transformers import ViTFeatureExtractor, ViTModel

In [3]:
# model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [4]:
# feature_extractor

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
import os

In [8]:
import torch
from torch import nn

In [9]:
from torch.nn.utils.rnn import pad_sequence

In [10]:
class MusicDataset(torch.utils.data.Dataset):   
    def __init__(self, np_file_paths, labels, seq_len=10000):
        self.seq_len = seq_len
        self.files = np_file_paths
        self.padder = torch.zeros(96, seq_len)
        self.labels = labels
#         self.labels = []
#         for i in range(len(self.files)):
#             label = np.random.randint(0, 10, size=15)
#             label[label > 8] = 0
#             label[label >= 1] = 1
#             self.labels.append(label)
#         for i in range(len(self.files)):
#             label = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
#             self.labels.append(label)
    def __len__(self):
        return len(self.files)

#     def __getitem__(self, index):
#         return (self.X[index:index+self.seq_len], self.y[index+self.seq_len-1])
    def __getitem__(self, index):
        x = np.load(self.files[index])
        x = torch.from_numpy(x).float()
        x = x[:,:self.seq_len]
        x = pad_sequence([x.T, self.padder.T], padding_value=-90, batch_first=True)[0].T
#         input,label_ids,label
        item = {"input": x, "label_ids":[index], "labels": torch.tensor(self.labels[index])}
        return item
#         return x

In [11]:
from datasets import load_dataset

In [12]:
test_files = ['00/1164200.mp3',
 '00/12100.mp3',
 '00/1295900.mp3',
 '00/985000.mp3',
 '00/1398500.mp3',
 '00/1210700.mp3',
 '00/818600.mp3',
 '00/1339600.mp3',
 '00/506100.mp3',
 '00/390000.mp3',
 '00/16000.mp3',
 '00/1052800.mp3',
 '00/699100.mp3']

In [13]:
good_files = ['01/16101.mp3',
 '01/1052801.mp3',
 '01/12101.mp3',
 '01/1121101.mp3',
 '01/986601.mp3',
 '01/1125001.mp3',
 '01/1086601.mp3',
 '01/1219101.mp3',
 '01/759301.mp3',
 '01/1018801.mp3',
 '01/824301.mp3',
 '01/1167301.mp3',
 '01/1380601.mp3',
 '01/661601.mp3',
 '01/1398501.mp3',
 '01/390001.mp3',
 '01/80501.mp3',
 '01/1125401.mp3',
 '01/399201.mp3',
 '01/1210701.mp3',
 '01/554901.mp3',
 '01/292501.mp3',
 '01/842401.mp3',
 '01/1157701.mp3',
 '01/1245901.mp3',
 '01/1062501.mp3',
 '01/1189901.mp3',
 '01/1398801.mp3',
 '01/1357701.mp3',
 '01/1164201.mp3',
 '01/1396501.mp3',
 '01/1304001.mp3',
 '01/913701.mp3',
 '01/718301.mp3',
 '01/1381001.mp3',
 '01/1264201.mp3',
 '01/361701.mp3',
 '01/1420701.mp3',
 '01/1406401.mp3',
 '01/708401.mp3',
 '01/1009701.mp3',
 '01/846501.mp3',]

In [14]:
def files_to_labels(files):
    col_names = ['TRACK_ID',
     'ARTIST_ID',
     'ALBUM_ID',
     'PATH',
     'DURATION',
     'TAGS',
     'TAGS2',
     'TAGS3',
     'TAGS4',
     'TAGS5',
     'TAGS6',
     'TAGS7',
     'TAGS8',
     'TAGS9']
    MOODPATH = "/mnt/c/Users/aag12/Downloads/autotagging_moodtheme.tsv.txt"
    df = pd.read_csv(MOODPATH, sep='\t', names=col_names)
    df = df[df["PATH"].isin(files)]
    inds = {'fast': 0,
     'sexy': 1,
     'mellow': 2,
     'heavy': 3,
     'horror': 4,
     'travel': 5,
     'holiday': 6,
     'groovy': 7,
     'funny': 8,
     'retro': 9,
     'hopeful': 10,
     'powerful': 11,
     'cool': 12,
     'nature': 13,
     'game': 14}

    final_labels = []
    for i in range(len(df)):
        curr = np.zeros(len(inds))
        moods = list(df.iloc[i])[5:]
        for theme in moods:
            if type(theme) == str and "mood" in theme:
                check = theme.split("---")[-1]
                if check in inds:
                    curr[inds[check]] = 1
            else:
                pass

        final_labels.append(curr)
    return final_labels

In [15]:
train_labels = files_to_labels(good_files)

In [16]:
test_labels = files_to_labels(test_files)

In [17]:
train_np_files = [("/mnt/c/Users/aag12/Documents/subset_moodtheme/" + g).replace(".mp3", ".npy") for g in good_files]
test_np_files = [("/mnt/c/Users/aag12/Documents/subset_moodtheme/" + g).replace(".mp3", ".npy") for g in test_files]

In [18]:
import time

In [19]:
# SEQ_LEN = 10000
SEQ_LEN = 1000

In [20]:
train_dataset = MusicDataset(train_np_files, train_labels, seq_len=SEQ_LEN)
test_dataset = MusicDataset(test_np_files, test_labels, seq_len=SEQ_LEN)

In [21]:
len(train_dataset), len(test_dataset)

(42, 13)

In [22]:
from transformers import Trainer, TrainingArguments

In [23]:
# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=1,  # batch size per device during training
#     per_device_eval_batch_size=1,   # batch size for evaluation
#     warmup_steps=10,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

In [24]:
class Flatten(torch.nn.Module):
    def forward(self, x):
#         batch_size = x.shape[0]
#         return x.view(batch_size, -1)
#         return x.view(batch_size, -1)
#         print(x.shape)
#         print(x.flatten().shape)
        return x.flatten()

In [25]:
# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=dataset,         # training dataset
#     eval_dataset=dataset             # evaluation dataset
# )

In [26]:
train_dataset[0]['input'].shape

torch.Size([96, 1000])

In [27]:
# trainer.train()

In [28]:
from torch.utils.data import DataLoader
# from transformers import AdamW
import torch.optim as optim

In [29]:
model = nn.Sequential(
        Flatten(),
        nn.Linear(96*SEQ_LEN, 512),
        nn.ReLU(),
#         nn.Linear(1000, 512),
#         nn.ReLU(),
#         nn.Linear(1024, 512),
#         nn.Linear(512, 124),
#         nn.Softmax(),
#             nn.Sigmoid(),
        )
heads = [nn.Sequential(nn.Linear(512, 2), nn.Sigmoid()) for i in range(15)]

In [39]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
# criterion = nn.CrossEntropyLoss()
losses = [nn.CrossEntropyLoss() for i in range(15)]

In [44]:
int(labels[i])

0

In [48]:
model = model.to(device)
heads = [h.to(device) for h in heads]
model.train()
for epoch in range(5):
    total_loss = 0
#     for batch in train_loader:
        
    for batch in train_dataset:
        optimizer.zero_grad()
        inputs = batch['input'].to(device)
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(inputs)
        per_head_outputs = [heads[i](outputs) for i in range(15)]
        these_losses = []
        for i in range(15):
            target = torch.tensor([0, 0])
            target[int(labels[i])] = 1
            target = target.to(device)
            cur_loss = losses[i](per_head_outputs[i], target.float())
            these_losses.append(cur_loss)
#         loss = outputs[0]
#         loss = criterion(outputs, labels)
        loss = sum(these_losses)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch: {epoch}; Loss: {total_loss}")

model.eval()

Epoch: 0; Loss: 411.43996238708496
Epoch: 1; Loss: 409.0412788391113
Epoch: 2; Loss: 409.0412788391113
Epoch: 3; Loss: 409.0412788391113
Epoch: 4; Loss: 409.0412788391113


Sequential(
  (0): Flatten()
  (1): Linear(in_features=96000, out_features=512, bias=True)
  (2): ReLU()
)

In [43]:
nn.functional.softmax(model(batch['input'].to(device))).sum()

/tmp/ipykernel_1454/3039809088.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.functional.softmax(model(batch['input'].to(device))).sum()


tensor(1., device='cuda:0', grad_fn=<SumBackward0>)

In [45]:
model(batch['input'].to(device))

tensor([-0.0255, -0.0454, -0.0275,  0.0358, -0.1361, -0.0021, -0.0321,  0.0438,
         0.0119,  0.0229,  0.0430,  0.0252,  0.0128,  0.0323, -0.0048],
       device='cuda:0', grad_fn=<AddBackward0>)

In [41]:
model(batch['input'].to(device))

tensor([-0.0255, -0.0454, -0.0275,  0.0358, -0.1361, -0.0021, -0.0321,  0.0438,
         0.0119,  0.0229,  0.0430,  0.0252,  0.0128,  0.0323, -0.0048],
       device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
batch['input'].to(device)

In [34]:
with torch.no_grad():
#     a = model(dataset[2]['input'].to(device))
    total_correct = 0
    total_values = 0
    for batch in test_dataset:
        outputs = model(batch['input'].to(device))
        labels = batch['labels']
#         outputs = nn.Softmax()(outputs)
        outputs[outputs > 0] = 1
        outputs[outputs < 0] = 0
        total_correct += (labels == outputs.to('cpu')).sum()
        total_values += len(labels)
    

In [38]:
1/15

0.06666666666666667

In [36]:
total_correct / total_values

tensor(0.4923)

In [37]:
with torch.no_grad():
#     a = model(dataset[2]['input'].to(device))
    total_correct = 0
    total_values = 0
    for batch in train_dataset:
        outputs = model(batch['input'].to(device))
        labels = batch['labels']
#         outputs = nn.Softmax()(outputs)
        outputs[outputs > 0] = 1
        outputs[outputs < 0] = 0
        total_correct += (labels == outputs.to('cpu')).sum()
        total_values += len(labels)
    

In [39]:
total_correct / total_values

tensor(0.5127)

In [272]:
with torch.no_grad():
    for batch in train_loader:
        inputs = batch['input'].to(device)
        outputs = model(inputs)
        

In [273]:
import gc
torch.cuda.empty_cache()
gc.collect()

1875

In [274]:
from torch import nn

In [140]:
transformer_model = nn.Transformer(nhead=16, num_encoder_layers=12)
src = torch.rand((10, 32, 512))
tgt = torch.rand((20, 32, 512))
out = transformer_model(src, tgt)

In [141]:
out

tensor([[[-1.5214,  0.7462,  1.5416,  ...,  1.1933,  0.6153, -0.0659],
         [-1.4950,  1.2451,  0.5920,  ..., -0.0223,  0.9517, -0.6634],
         [-0.6652,  0.8013,  1.3819,  ...,  0.7682,  0.2696,  0.0530],
         ...,
         [-2.1992,  0.7358,  1.9855,  ...,  0.0718,  1.0148, -0.1242],
         [-1.6068,  0.0923,  1.7142,  ...,  1.2601, -0.2076, -0.1038],
         [-1.2549, -0.3516,  0.8073,  ...,  0.2366,  0.7856,  0.1373]],

        [[-1.5566,  0.8384,  1.3449,  ...,  0.6766,  0.3548,  0.7389],
         [-0.8511,  0.8868,  0.6050,  ..., -0.6807,  1.2467, -1.2463],
         [-1.1151,  0.3095,  0.8905,  ...,  0.1912,  0.2807,  0.5792],
         ...,
         [-1.6557,  0.0951,  0.9750,  ...,  0.6561,  0.4272,  0.1767],
         [-1.6226, -0.1198,  1.5999,  ...,  0.4322,  1.4955, -0.1060],
         [-1.7911,  0.3468,  0.4423,  ...,  0.5350,  0.9664, -0.7834]],

        [[-1.1344,  1.3616,  2.0517,  ...,  0.0625,  0.2578, -0.8662],
         [-1.4028,  0.0594,  0.2899,  ...,  0